In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install -U bitsandbytes transformers

In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
!pip install -U langchain-community


In [ ]:
!pip install datasets

In [5]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_path = "/content/drive/MyDrive/"

In [ ]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# 데이터 분할
train_data, test_data = train_test_split(train, test_size=0.2, random_state=42)

# 훈련 데이터 변환
train_transformed = train_data.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# 테스트 데이터 변환
test_transformed = test_data.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
train_df = pd.DataFrame(list(train_transformed))
test_df = pd.DataFrame(list(test_transformed))

# 확인
print(train_df.head())
print(test_df.head())


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True  # 8bit 양자화 활성화
)

In [ ]:
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

In [ ]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm

# Train 데이터 준비
train_questions_prevention = train_df['question'].tolist()
train_answers_prevention = train_df['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text=False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

Device set to use cuda:0


In [ ]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

<ipython-input-17-8844173da8bf>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
tokenizer.padding_side = 'left'

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset
import torch
from tqdm import tqdm
import psutil

max_num_workers = psutil.cpu_count() - 1  # CPU 코어 수 - 1 (시스템 안정성을 위해 1개 예약)
min_batch_size = 4  # 최소 배치 크기
max_batch_size = 64  # 최대 배치 크기 (GPU 메모리에 따라 조정 가능)

# Matmul 성능 최적화
torch.set_float32_matmul_precision('high')

# 최적의 num_workers 계산
def calculate_optimal_num_workers():
    cpu_usage = psutil.cpu_percent(interval=1)
    available_cores = max_num_workers
    # CPU 사용률이 80% 이상이면 num_workers 줄임
    if cpu_usage > 80:
        return max(1, available_cores // 2)
    return available_cores

# 테스트 데이터 준비
test_questions = test_df['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})

# DataLoader 설정 (num_workers로 병렬 처리 활성화)
batch_size = 16
num_workers = calculate_optimal_num_workers()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# 배치 처리 함수
def process_batch(batch):
    contexts = [retriever.invoke(q)[0].page_content for q in batch["question"]]
    prompts = [prompt.format(context=c, question=q) for c, q in zip(contexts, batch["question"])]
    results = text_generation_pipeline(prompts, batch_size=batch_size)
    return [r[0]["generated_text"] for r in results]

# DataLoader로 병렬 처리
for batch in tqdm(test_dataloader, desc="처리 진행률"):
    batch_results = process_batch(batch)
    test_results.extend(batch_results)
print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 4685


처리 진행률:   0%|          | 0/293 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
처리 진행률:   0%|          | 1/293 [00:08<42:50,  8.80s/it]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
처리 진행률:   1%|          | 2/293 [00:19<48:53, 10.08s/it]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
처리 진행률


테스트 실행 완료! 총 결과 수: 4685


In [ ]:
test_results

['모터 교체 작업 시 필요한 받침대를 사전에 설치하고, 작업 전에 모든 작업자에게 받침대의 중요성을 교육하며 정기적인 안전 점검을 실시합니다.',
 '안전고리 체결 의무화 및 정기적인 안전 점검 강화를 통해 재발 방지 대책을 마련하고, 모든 고소작업 전 안전 교육을 의무화하여 향후 조치 계획을 수립합니다.',
 '결빙 구간에 미끄럼 방지 매트 설치하고, 작업자들에게 안전 교육 강화하여 결빙 구간 인지 및 주의를 촉구합니다.',
 '개인 안전 교육 강화 및 주의사항 준수 점검 실시.',
 '심장마비 재발 방지를 위해 정기적인 건강검진과 스트레스 관리 교육을 실시하고, 작업 환경 개선을 통해 근로자의 건강 상태를 지속적으로 모니터링하겠습니다.',
 '정기적인 건강 검진 실시와 개인 맞춤형 안전 교육 강화를 통해 근로자의 기저 질환 관리 및 작업 적합성 평가를 병행합니다.',
 '작업자 건강 모니터링 강화 및 폭염 시 작업 제한 지침 마련.',
 '안전장치 점검 강화 및 안전 교육 강화 계획 수립.',
 '안전 발판 설치 의무화 및 정기적인 미끄럼 방지 매트 점검 실시, 이동 중 안전 교육 강화',
 '안전 교육 강화 및 지속적인 감독 체계 구축으로 부주의 사고 예방. 정기적 안전 점검 실시 및 안전 수칙 준수 의무화.',
 '안전 교육 강화와 개인 보호 장비 착용 의무화를 통해 재발 방지 대책을 마련하고, 정기적인 안전 점검과 사고 예방 훈련을 실시할 계획입니다.',
 '안전 교육 강화와 함께 이동식 크레인 조종사 및 신호자 간의 명확한 의사소통 프로토콜 도입, 모든 고소작업 시 안전대 착용 의무화 및 정기적인 안전 점검 실시.',
 '안전 교육 강화 및 정기적인 안전 점검 실시.',
 '장비 점검 강화와 정기적인 유지보수 실시를 통한 재발 방지 대책. 향후에는 사전 예방 점검을 강화하고, 장비 제조사와 협력하여 안전 기준을 엄격히 준수할 계획입니다.',
 '작업 전 안전 교육 강화 및 추락 위험 구역에 대한 안전 점검 체계 구축을 포함한 재발 방지 대책 및 향

In [ ]:
test_results[0]

'모터 교체 작업 시 필요한 받침대를 사전에 설치하고, 작업 전에 모든 작업자에게 받침대의 중요성을 교육하며 정기적인 안전 점검을 실시합니다.'

In [ ]:
import pandas as pd

# test_results가 리스트 또는 Series라고 가정
result_df = pd.DataFrame({
    "predicted_answer": test_results,
    "actual_answer": test_df["answer"].values  # 실제 정답
})

# CSV 파일로 저장
result_df.to_csv("test_results2.csv", index=False, encoding="utf-8-sig")

# 확인
print(result_df.head())


                                    predicted_answer  \
0  모터 교체 작업 시 필요한 받침대를 사전에 설치하고, 작업 전에 모든 작업자에게 받...   
1  안전고리 체결 의무화 및 정기적인 안전 점검 강화를 통해 재발 방지 대책을 마련하고...   
2  결빙 구간에 미끄럼 방지 매트 설치하고, 작업자들에게 안전 교육 강화하여 결빙 구간...   
3                       개인 안전 교육 강화 및 주의사항 준수 점검 실시.   
4  심장마비 재발 방지를 위해 정기적인 건강검진과 스트레스 관리 교육을 실시하고, 작업...   

                                       actual_answer  
0  모터 교체 작업 시 모터를 내려놓을 곳에 미리 받침대를 설치하여 모터가 땅에 떨어지...  
1                작업전 안전교육 철저에 대한 재발 방지 대책과 향후 조치 계획.  
2                     옥외구간 이동 시 지정된 통로(부직포 설치구간) 이용.  
3                      단철근 절단 작업 시 소형 핸드컷터 사용 권장 사항.  
4                       현장보존을 통한 재발 방지 대책과 향후 조치 계획.  


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [3]:
# 코사인 유사도 함수
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

# 점수 계산 함수
def calculate_scores(df):
    cosine_scores = []
    jaccard_scores = []

    for i in range(len(df)):
        sentence1 = df["predicted_answer"].iloc[i]
        sentence2 = df["actual_answer"].iloc[i]

        # 문장 임베딩
        embedding1 = model.encode(sentence1, convert_to_numpy=True)
        embedding2 = model.encode(sentence2, convert_to_numpy=True)

        # 유사도 계산
        cosine_sim = cosine_similarity(embedding1, embedding2)
        jaccard_sim = jaccard_similarity(sentence1, sentence2)

        # 리스트에 저장
        cosine_scores.append(cosine_sim)
        jaccard_scores.append(jaccard_sim)

    # 새로운 컬럼 추가
    df["cosine_similarity"] = cosine_scores
    df["jaccard_similarity"] = jaccard_scores

    # 전체 평균 계산
    avg_cosine = np.mean(cosine_scores)
    avg_jaccard = np.mean(jaccard_scores)

    print(f"전체 평균 코사인 유사도: {avg_cosine:.4f}")
    print(f"전체 평균 자카드 유사도: {avg_jaccard:.4f}")

    return df


# 데이터프레임에 유사도 추가
#result_df = calculate_scores(result_df)



In [16]:
df1 = pd.read_csv("C:/Users/minkyu/Desktop/open/submissionV8.csv")
#df2 = pd.read_csv("C:/Users/minkyu/Desktop/open/submissionV9.csv")
# 두 DataFrame의 비교할 컬럼을 하나의 DataFrame으로 합칩니다.
compare_df = pd.DataFrame({
    "predicted_answer": df1['재발방지대책 및 향후조치계획'],
    "actual_answer": df2['재발방지대책 및 향후조치계획']
})

# calculate_scores 함수를 이용해 유사도 계산
compare_df = calculate_scores(compare_df)


전체 평균 코사인 유사도: 0.5827
전체 평균 자카드 유사도: 0.1996


In [17]:
0.5827 *0.7 + 0.1996*0.3

0.46776999999999996

In [15]:
df2['재발방지대책 및 향후조치계획'] = '안전교육을 통한 재발 방지 대책 및 향후 조치 계획'


In [12]:
df2

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,"펌프카 아웃트리거 보강 강화 및 지반 안정성 검토, 장비 운용 교육 강화 및 작업 ...",-1.088458,-0.806697,0.067113,0.680635,-0.764054,0.947974,1.042896,-0.079980,...,0.951112,0.860278,-0.000003,1.438590,0.215181,0.041279,-0.291659,-0.210333,0.659329,-0.182199
1,TEST_001,작업 전 철저한 안전교육 실시와 보안면 착용 의무화 강화를 통해 재발 방지 대책 마련.,-0.884666,0.459320,-0.007384,-0.038709,-0.724180,0.488116,0.822417,0.398957,...,0.339614,1.897606,1.035079,1.781253,0.933693,-0.323275,0.140578,-0.376626,0.901251,-0.095426
2,TEST_002,작업자에게 이동 중 전방 주의 의무 강화 교육 실시 및 계단 주변 안전 표시 설치.,-0.777984,0.500808,0.054636,0.331050,-0.434892,0.775530,0.476251,-0.104884,...,0.396288,1.564759,-0.550334,0.978587,1.361517,-0.659481,-0.284633,-0.061770,0.384578,0.524505
3,TEST_003,"작업 공간의 정기적인 청소와 안전한 작업 발판 설치를 강화하고, 근로자들에게 안전 ...",-0.335416,0.517941,-0.619364,0.609516,-0.240747,0.914001,-0.236455,0.035741,...,0.594422,1.186995,0.312634,0.688071,0.440973,-0.201366,0.262706,-0.134687,0.911749,-0.099271
4,TEST_004,작업자 안전 교육 강화 및 이동 중 안전 수칙 준수 강조를 통해 재발 방지. 향후 ...,-0.718897,0.313774,-0.005176,0.699394,-0.280992,0.958968,-0.260164,-0.458979,...,0.488002,1.781263,0.421615,1.715215,1.473447,-1.098084,-0.685962,-0.096680,0.687249,-0.271030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,TEST_959,작업 중 약한 석재의 안전한 처리 방법 교육 강화와 정기적인 안전 점검 실시를 통한...,0.195847,0.065073,-0.108859,0.493188,-0.369775,0.883152,0.164443,-0.304190,...,0.816511,0.814625,0.174905,2.046105,1.141846,-0.224764,0.714994,-0.099591,1.075362,-0.248021
960,TEST_960,근로자 이동 통로 확보 의무화 및 안전 교육 강화를 통해 재발 방지 대책을 마련하고...,-0.129197,0.721665,-0.465958,1.113835,-0.858985,0.896192,-0.080010,0.393469,...,0.458626,1.196076,-0.542913,1.019144,0.524694,-0.367742,-0.283518,-0.106402,0.894998,-0.056406
961,TEST_961,작업자 안전 교육 강화 및 작업 공간 안전 점검 실시를 통한 재발 방지 대책 및 향...,-0.015337,-0.069063,0.109485,0.304157,-0.086632,0.463580,0.154031,-0.152738,...,0.272521,1.496647,0.381519,1.515937,0.605126,-0.367574,-0.058026,-0.074111,1.403812,-0.125771
962,TEST_962,안전교육 강화와 작업 전 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획 수립.,-0.127954,0.074774,0.281000,0.065185,0.032348,0.490508,0.381698,-0.456348,...,0.413578,1.648472,0.860277,2.103032,0.912913,-0.628851,0.232860,-0.013360,1.056469,-0.226130
